In [3]:
# !pip install -r requirements.txt

In [4]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import logging
import multiprocessing
from datetime import datetime, timedelta

from src import modules as f
%load_ext autoreload
%autoreload 2

In [39]:
stock_list = f.download_interval_process('1d', 1)

In [40]:
stock_list

{'AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AEP',
 'AJG',
 'AMGN',
 'AMZN',
 'APH',
 'ATO',
 'AVGO',
 'AXP',
 'BAC',
 'BRK-B',
 'BSX',
 'CEG',
 'CL',
 'CMCSA',
 'COP',
 'COST',
 'CRM',
 'CSCO',
 'CVX',
 'D',
 'DASH',
 'DECK',
 'DIS',
 'DUK',
 'EOG',
 'EXC',
 'EXE',
 'FCNCA',
 'GOOG',
 'GOOGL',
 'GS',
 'HD',
 'IBM',
 'ISRG',
 'JNJ',
 'JPM',
 'KMI',
 'KO',
 'LLY',
 'LNG',
 'MA',
 'MDLZ',
 'META',
 'MO',
 'MPC',
 'MRK',
 'MS',
 'MSFT',
 'MSI',
 'NEE',
 'NFLX',
 'NOW',
 'NRG',
 'NTRA',
 'NVDA',
 'OKE',
 'ORCL',
 'PCAR',
 'PEG',
 'PEP',
 'PG',
 'PLTR',
 'PM',
 'PSTG',
 'PTC',
 'RCL',
 'SLB',
 'SO',
 'SPGI',
 'SRE',
 'SW',
 'T',
 'TDG',
 'TGT',
 'TMO',
 'TMUS',
 'TSLA',
 'TTD',
 'UNH',
 'V',
 'VST',
 'VZ',
 'WELL',
 'WFC',
 'WMB',
 'WMT',
 'WSM',
 'XOM',
 '^VIX'}

In [41]:
stock_list = f.make_table_features_process('1d', 8)

Error fetching data for VOO: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v10/finance/quoteSummary/VOO?modules=quoteType%2CsummaryProfile%2CtopHoldings%2CfundProfile&corsDomain=finance.yahoo.com&symbol=VOO&formatted=false&crumb=tTRVe6ysdIp (Caused by SSLError(SSLError(1, '[SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2559)')))


In [42]:
f.make_master_table(stock_list, interval = '1d')

Saved combined dataframe to: ./data_transformed/all_1d_model_df.pkl


In [43]:
all_1d_df = f.load_model_df('all', '1d')
all_1d_df.sample(25)

,open,high,low,close,adj_close,volume,slow_sma_signal,fast_sma_signal,stdev20,stdev10,...,bottom_stdev10,bottom_stdev5,pct_gap_up_down_stdev20,pct_gap_up_down_stdev10,pct_gap_up_down_stdev5,month_of_year,day_of_month,day_of_week,hour_of_day,direction
333181,122.650002,122.650002,117.910004,118.029999,105.945068,8600,1,0,1.247422,0.710671,...,0.242986,0.150653,0.538916,0.568869,0.683792,7,21,2,0,2
433968,0.151572,0.153498,0.151572,0.153498,0.099002,2997263,1,1,0.003486,0.004359,...,0.286652,0.055880,0.666763,0.527012,0.290189,12,14,1,0,2
149303,233.539993,234.600006,231.720001,234.029999,234.029999,4850400,1,1,3.637018,2.396971,...,0.169283,0.177870,0.428368,0.337069,0.303619,1,8,4,0,0
174124,25.360001,26.320000,25.360001,25.580000,25.580000,2347700,1,0,0.894756,0.762455,...,0.176033,0.160984,0.489425,0.582432,0.530828,2,22,1,0,2
426284,51.500000,51.680000,50.970001,51.459999,39.859814,3965000,1,1,0.797474,0.539924,...,0.211185,0.225025,0.377017,0.310266,0.270788,5,9,1,0,1
566766,52.570000,52.889999,52.389999,52.669998,35.975323,28935600,1,1,1.691891,0.668000,...,0.235712,0.215784,0.456402,0.557665,0.665129,12,15,4,0,1
146331,55.500000,56.139999,55.060001,55.060001,55.060001,1450000,1,1,3.487228,2.810505,...,0.196727,0.141495,0.488653,0.365154,0.350191,3,20,4,0,1
519905,17.000000,17.000000,16.875000,16.875000,0.663570,53900,1,1,0.015985,0.011826,...,0.331593,0.235702,0.096546,0.000000,0.000000,4,3,3,0,0
440585,12.865604,13.030021,12.660083,13.019745,9.556556,8855550,1,1,0.171864,0.225754,...,0.193654,0.250546,0.039841,0.038332,0.054209,4,3,4,0,0
329381,0.000000,28.500000,28.500000,28.500000,20.917435,3000,0,1,0.295631,0.289471,...,10.164464,14.374723,17.813343,10.480692,14.821936,6,26,0,0,0


In [44]:
f.xg_boost_model()

/home/oem/Documents/github/stock_production_agg/stkenv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [11:38:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       0.39      0.37      0.38     54930
           1       0.41      0.42      0.42     54940
           2       0.41      0.42      0.42     54938

    accuracy                           0.40    164808
   macro avg       0.40      0.40      0.40    164808
weighted avg       0.40      0.40      0.40    164808


Feature Importances (Descending):
                   feature  importance
38    cat__day_of_month_23    0.028203
42    cat__day_of_month_27    0.028153
28    cat__day_of_month_13    0.026432
46    cat__day_of_month_31    0.025706
45    cat__day_of_month_30    0.025584
19     cat__day_of_month_4    0.024788
13   cat__month_of_year_10    0.024684
36    cat__day_of_month_21    0.024659
14   cat__month_of_year_11    0.024650
15   cat__month_of_year_12    0.024476
10    cat__month_of_year_7    0.024148
11    cat__month_of_year_8    0.023951
23     cat__day_of_month_8    0.023938
33    cat__day_of_month_18    0.023884